# TYTX - Struct Types

**Structs** allow you to define schemas for complex data structures, with support for:
- Nested types
- Typed arrays
- References between structs
- Metadata (validation, UI hints)

In [ ]:
from decimal import Decimal
from datetime import date

from genro_tytx import registry, from_json, as_typed_json

## 1. Registering a Struct

A struct defines the structure of an object using TYTX type codes.

In [ ]:
# Register the CUSTOMER struct
registry.register_struct(
    "CUSTOMER",
    '{"name": "T", "balance": "N", "active": "B"}'
)

print("Struct CUSTOMER registered!")
print(f"Schema: {registry.get_struct('CUSTOMER')}")

## 2. Hydrating Data with Struct

When receiving JSON data with a struct reference, TYTX automatically hydrates the types.

In [ ]:
# Raw JSON data (all strings/native numbers)
json_data = '''{
    "name": "John Doe",
    "balance": "1500.50",
    "active": "true"
}::@CUSTOMER'''

# Parse with hydration
from genro_tytx import from_text

customer = from_text(json_data)

print("Hydrated data:")
for k, v in customer.items():
    print(f"  {k}: {v} ({type(v).__name__})")

## 3. Nested Structs

Structs can reference other structs using the `@` prefix.

In [ ]:
# Register ADDRESS struct
registry.register_struct(
    "ADDRESS",
    '{"street": "T", "city": "T", "zip": "T"}'
)

# Register ORDER that references CUSTOMER
registry.register_struct(
    "ORDER",
    '{"id": "L", "customer": "@CUSTOMER", "total": "N", "date": "D"}'
)

print("Structs registered:")
print(f"  ADDRESS: {registry.get_struct('ADDRESS')}")
print(f"  ORDER: {registry.get_struct('ORDER')}")

In [ ]:
# JSON data with nested struct
order_json = '''{
    "id": "12345",
    "customer": {
        "name": "Jane Smith",
        "balance": "3000.00",
        "active": "true"
    },
    "total": "299.99",
    "date": "2025-12-04"
}::@ORDER'''

order = from_text(order_json)

print("Hydrated order:")
print(f"  ID: {order['id']} ({type(order['id']).__name__})")
print(f"  Customer: {order['customer']['name']}")
print(f"  Customer balance: {order['customer']['balance']} ({type(order['customer']['balance']).__name__})")
print(f"  Total: {order['total']} ({type(order['total']).__name__})")
print(f"  Date: {order['date']} ({type(order['date']).__name__})")

## 4. Typed Arrays

TYTX supports homogeneous arrays with the `#` prefix.

In [ ]:
# Array of integers
int_array = from_text('[1, 2, 3, 4, 5]::#L')
print(f"Integer array: {int_array}")
print(f"Element type: {type(int_array[0]).__name__}")

In [ ]:
# Array of Decimals
decimal_array = from_text('["10.5", "20.75", "30.99"]::#N')
print(f"Decimal array: {decimal_array}")
print(f"Element type: {type(decimal_array[0]).__name__}")

In [ ]:
# Array of structs!
customers_json = '''[
    {"name": "Alice", "balance": "100.00", "active": "true"},
    {"name": "Bob", "balance": "200.00", "active": "false"}
]::#@CUSTOMER'''

customers = from_text(customers_json)

print("Customer array:")
for c in customers:
    print(f"  {c['name']}: {c['balance']} ({type(c['balance']).__name__}) - active: {c['active']}")

## 5. List Schema (Positional)

Structs can be defined as lists for tabular data.

In [ ]:
# Positional schema: [name, quantity, price]
registry.register_struct(
    "ROW",
    '["T", "L", "N"]'
)

# Tabular data
row_json = '["Product A", "5", "29.99"]::@ROW'
row = from_text(row_json)

print(f"Row: {row}")
print(f"  Name: {row[0]} ({type(row[0]).__name__})")
print(f"  Quantity: {row[1]} ({type(row[1]).__name__})")
print(f"  Price: {row[2]} ({type(row[2]).__name__})")

In [ ]:
# Array of rows
table_json = '''[
    ["Apple", "10", "1.50"],
    ["Pear", "5", "2.00"],
    ["Banana", "8", "1.20"]
]::#@ROW'''

table = from_text(table_json)

print("Product table:")
print(f"{'Name':<10} {'Qty':>5} {'Price':>10}")
print("-" * 27)
for row in table:
    print(f"{row[0]:<10} {row[1]:>5} {row[2]:>10}")

## 6. Struct with Metadata

Metadata allows adding validation and UI information.

In [ ]:
# Register struct with metadata
registry.register_struct(
    "PRODUCT",
    schema='{"code": "T", "name": "T", "price": "N", "stock": "L"}',
    metadata={
        "code": {
            "validate": {"min": 3, "max": 10, "required": True},
            "ui": {"label": "Product Code"}
        },
        "name": {
            "validate": {"min": 1, "max": 200},
            "ui": {"label": "Name", "placeholder": "Enter product name"}
        },
        "price": {
            "validate": {"min": 0},
            "ui": {"label": "Price", "format": "currency"}
        },
        "stock": {
            "validate": {"min": 0, "default": 0},
            "ui": {"label": "Stock"}
        }
    }
)

print("PRODUCT schema:", registry.get_struct("PRODUCT"))

In [ ]:
# Retrieve metadata
all_meta = registry.get_struct_metadata("PRODUCT")
print("All metadata:")
for field, meta in all_meta.items():
    print(f"  {field}: {meta}")

In [ ]:
# Single field metadata
price_meta = registry.get_struct_metadata("PRODUCT", "price")
print(f"'price' metadata: {price_meta}")

## 7. Passthrough (no conversion)

Use empty string `""` for fields that don't require conversion.

In [ ]:
# Schema with passthrough for JSON-native types
registry.register_struct(
    "MIXED",
    '{"name": "", "active": "", "balance": "N"}'
)

mixed_json = '''{
    "name": "Test",
    "active": true,
    "balance": "100.50"
}::@MIXED'''

mixed = from_text(mixed_json)

print("Mixed data:")
for k, v in mixed.items():
    print(f"  {k}: {v} ({type(v).__name__})")

print("\nNote: 'active' and 'name' keep their JSON-native types")

## 8. Inline Nested Struct

You can define nested structs inline without registering them separately.

In [ ]:
# Struct with inline nested
registry.register_struct(
    "INVOICE",
    '''{
        "number": "T",
        "date": "D",
        "customer": {"name": "T", "vat": "T"},
        "total": "N"
    }'''
)

invoice_json = '''{
    "number": "INV-001",
    "date": "2025-12-04",
    "customer": {"name": "Acme Corp", "vat": "US123456789"},
    "total": "1500.00"
}::@INVOICE'''

invoice = from_text(invoice_json)

print(f"Invoice: {invoice['number']}")
print(f"Date: {invoice['date']} ({type(invoice['date']).__name__})")
print(f"Customer: {invoice['customer']['name']}")
print(f"Total: {invoice['total']} ({type(invoice['total']).__name__})")

## Summary

| Feature | Syntax | Example |
|---------|--------|---------|
| Dict struct | `{"field": "TYPE"}` | `{"name": "T", "age": "L"}` |
| List struct | `["TYPE1", "TYPE2"]` | `["T", "L", "N"]` |
| Struct ref | `@CODE` | `"customer": "@CUSTOMER"` |
| Typed array | `#TYPE` | `[1,2,3]::#L` |
| Struct array | `#@CODE` | `[...]::#@CUSTOMER` |
| Passthrough | `""` | `"active": ""` |
| Inline nested | `{...}` | `"addr": {"city": "T"}` |

## Next Notebook

In the next notebook we'll explore **XML support** for TYTX.